# Analysis with DuckDB SQL Queries

The following configuration allows to write queries in notebook-cells directly.

In [ ]:
import duckdb
import pandas as pd
import sqlalchemy

%load_ext sql

# Set a few config options to prettify the output and return it as Pandas DataFrame
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:
# DuckDB is primarily designed to be an in-memory DB. You can however persist your data to disk.
# %sql duckdb:///path/to/file.db

## Simple SQL queries

In [ ]:
%%sql

SELECT * FROM "./parquet_data/all_data.parquet/*.parquet" LIMIT 5;

Partitioned parquet data cannot be read in with DuckDB directly at the moment.

However, we can first read it in as arrow dataset and then query the arrow-table with DuckDB

```python
import pyarrow.dataset as ds
dataset = ds.dataset("./parquet_data/all_data.parquet/", format="parquet", partitioning="hive")

%%sql
SELECT * FROM dataset LIMIT 3;
```

## Return pandas dataframes for further processing

DuckDB query results can easily be converted to pandas dataframes which can then be further processed by pandas, for example for plots.

Interesting is that you can also query the pandas dataframe again with DuckDB SQL without any copying.

In [ ]:
import duckdb

top_producer = duckdb.query(
    """
    SELECT filter, SUM(production)/1e6 AS total_energy
    FROM './parquet_data/all_data.parquet/*.parquet'
    WHERE filter > 1000 AND filter < 4300
    GROUP BY filter
    ORDER BY total_energy DESC
    --LIMIT 5;
    """
).to_df()

In [ ]:
from config import FILTERS

top_producer["filter"] = top_producer["filter"].apply(lambda x: FILTERS[x])

top_producer

In [ ]:
type(top_producer)

In [ ]:
%%sql

SELECT * FROM top_producer LIMIT 2;

In [ ]:
top_producer.set_index("filter").plot.barh();